<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Chemicals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHEMICALS**

**Authors: Bridget Childs, Ananya Gangadhar, Jacqueline Wong**

[Monroe]: # (I'm realizing that you hadn't found all of my comments, so I've moved this out of the comments into the main text to ensure you find it. Make sure to open text boxes including this one! and the ones below to see if there are any comments!)

[Monroe]: # (The progress since the previous submission seems to be minimal. The analysis that you have done is both relatively simple and appears to have significant errors. Pick up the pace, review each other's work, bring healthy skepticism to your design, make your default position be that you DON'T believe what your team mates have done and make them convinced you. Design is so important because it is when we create the dream of how we are going to change the real world. You want to make sure that you are creating design recommendations that will work. Design isn't for a grade. Design mistakes are expensive at best and deadly at worst. That is why engineering skepticism is so important.)

[Monroe]: # (This was my feedback from the previous submission. You are making reasonable progress. Fix your estimates of flow rates and volumes, design the plumbing system, specify the pumps required, review your report for spelling errors [too bad that spell checkers don't work yet], design the storage tanks, etc.)



# **1. Task Definition**

<p align="center"> 
<img src="https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/chemicals_task.png">
</p>

The Chemicals team will be in charge of designing the chemical dosage system. This will include:
- Determining coagulant and polymer dosage
- Designing chemical storage, feed lines, and flow control system.
- Calculating feed hydraulics and plumbing
- Identifying available vendors.

Additionally, our storage system design will include the number of storage tanks, tank size, location, footprint, and safety add-ons in order to meet OSHA requirements (e.g. spillover capture, eye washes, safety showers). Information will be primarily sourced from government regulations, the Fairmont Sedimentation Plant Scope of Work document, relevant vendors, and similar go-by systems.  

# **2. Executive Summary**

(Highlight dominant constraints and your recommendation for the design.)

# **3. Constraints**

* Design flow of the plant is 720 cfs.
* Effluent arsenic levels should be under 10 ug/L.
* Effluent TOC levels should be under 3 mg/L.
* Effluent turbidity should be under 3 NTU.
* Chemicals will be stored in an open tank farm and have at least a 30 day capacity  (Scope of work, pg 18).
* Proposed tank farm should have area under 200'x100'
* All hazardous chemicals must be stored in an area that is designed to contain spills ([OSHA regulations](https://www.safetyinfo.com/safe-chemical-storage-osha-requirements-free-index/)).
* All design values should be justifiable using design equations.
* The chemical feed system should be easy to understand and operate, and should have relatively low maintenance.
* The designed system should have a feedback loop wherein the chemical dosage is adjusted accordingly with a change in the plant flow rate.

**CEE 4590 Logistical constraints**
* Information in Scope of Work is often missing citations, and we are unsure of the source and degree of certainty of parameters like expected FeCl3 dosage.
* At present, we have no communication with the LA engineers on this project, only the Stantec engineers, Mickey and Tori, who are working on Stantec's own proposal for this project. 
* It will be challenging to procure cost estimates from vendors since they are typically reluctant to share such information with students.
* It may be wise to add more redundancy to the chemical storage system than the recommended 30 days for situations like a shutdown due to a pandemic when supply chains are disrupted.

## **3.1. Table of potential strategies to meet the constraints and treatment objectives**

| strategy | advantages  | disadvantages | data needed |
|---|---|---|---|
| open tank farm vs. enclosed storage | go-by, less expensive | less protection | weathering risk |
| 3 storage tanks vs. using even more | less complexity | lower safety factor | storage area |
| choosing vendor before choosing tank size | easier to pick model | more restricted | vendor details |
| 30-day storage vs. keeping even more | resilience (e.g. COVID-19) | more expensive | area, budget |
| single vs. multiple dosage system per train | maintenance, operator ease | fewer options if pipe fails | clogging risk |

# **4. Physics**

Since the Chemicals Team task concerns mainly the storage and delivery of coagulant and polymers, it is important to consider the energy and space that these chemicals will occupy. The **Energy equation** accounts the energy losses in fluid mechanics. Below is the general form, which will be useful as it is tailored to the chemical dosage design. [(AguaClara Textbook)](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-review-review-fluid-mechanics-4)

$$\frac{p_{1}}{\rho g} + z_{1} + \alpha_{1} \frac{\bar v_{1}^2}{2g} + h_P = \frac{p_{2}}{\rho g} + z_{2} + {\alpha_{2}} \frac{\bar v_{2}^2}{2g} + h_T + h_L$$

Where:

p = pressure

ρ = density 

z = height 

v = velocity 

hp, ht = viscueous frictional loss due to pumps, turbines

hl = head loss


Head loss comprises of major losses from friction and minor losses from flow expansions, elbows. Major and minor loss equations are discussed below. 

 $$h_L = h_{\rm{f}} + h_e$$

Minor losses may be summarized as the equation below. Minor losses occur at flow expansions, elbows, and valves. K will vary depending on the kind of minor loss. 

$$h_e = \sum K \frac{\bar v_{out}^2}{2g} =\left( \frac{A_{out}}{A_{in}} -1 \right)^2 \frac{\bar v_{out}^2}{2g}$$

The **Darcy-Weisbach equation** solves for major losses in the head loss equation. Major losses comprise the frictional losses that occur in pipes. For this equation, it will be important to know the length and diameter of the pipes in our delivery system. Equation found in the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-darcy-weisbach). Additionally, there is an accompanying python function in the AguaClara python package to calculate major losses.

$$h_{\rm{f}} \, = \, {\rm{f}} \frac{L}{D} \frac{\bar v^2}{2g}$$

$$h_{\rm{f}} \, = \,{\rm{f}} \frac{8}{g \pi^2} \frac{LQ^2}{D^5}$$

Such that: 

hf = major loss 

f = Darcy friction factor
 
L = pipe length

Q = pipe flow rate
 
D = pipe diameter

AC Python Function = pc.headloss_fric(FlowRate, Diam, Length, Nu, PipeRough)


Other types of energy that may be of use to our chemical dosage system will be kinetic and potential energy, listed below. These are more general, but may be important to consider, especially with respect to the site's geography and elevation.   

$$ E_{potential} = mgh \,\,\,\,\, E_{kinetic} = mv^2/2$$

Ultimately, whatever power is neeeded to run the chemical dosage system will be supplied by pumps. Knowing the amount of power required will help choose the appropriate pump for the design. Below is the equation for power. E represents energy and t represents time. 

$$ P = \frac{E}{t} $$

Lastly, mass conservation will important to consider during our calculations. This will help us better understand how much storage will be required when considering the required dosage. 

$$Q_{in}C_{in} = Q_{out}C_{out}$$





# **5. Shared Assumptions**

The Chemical team will be utilizing the [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) file in conjunction with the other design teams in order to share assumptions from the Fairmont Sedimentation Tank Scope of Work document. It will also create dependency between the chemical design below and the designs of other teams.  

# **6. Major Decisions**

## **6.1. Ferric Chloride Coagulant**

The Fairmont Sedimentation Plant Scope of Work Document notes that two types of products will be used: a coagulant (specifically, ferric chloride) and emulsion polymer. Due to communication constraints with engineers at the Los Angeles Department of Water and Power, we are unlikely to fully understand the rationale behind the selection of ferric chloride as the coagulant. However, consider that it was already specified, we have decided to move forward with ferric chloride as the coagulant of choice.

Since the Scope of Work only states the average dose of ferric chloride is 13.14 mg/L. We have conservatively assumed that the maximum dose of ferric chloride will be 30 mg/L. This is slightly more than twice the average dose. 

## **6.2. Emulsion Polymer**

On the other hand, the type of emulsion polymer was not specified in the Scope of Work document. There are many types and blends of polymers that are available in the water treatment market, many of which are proprietary and are <del>vague</del>  in chemical makeup. 

Due to limitations of this design project, we will not be able to select an emulsion polymer with confidence without bench testing capabilities. However, we do know that the polymer is going to be cationic, and there are multiple vendors available. We will research a few options from different vendors and select a representative emulsion polymer to provide an estimate for density, and viscosity. More information about the representative polymer will be provided in the design section. 

## **6.3. Other Chemicals**

It was noted by other teams that a high coagulant dosage could significantly alter the pH of the water. Since the Scope of Work did not specify the need for other chemical products, we made the assumption that the acid neutralizing capacity of raw water entering the Fairmont Sedimentatino Plant is adequate, and that additional chemicals beyond the coagulant and emulsion polymer are not required.

## **6.4. Chemical Storage**

As both the Cottonwood Treatment Plant and Los Angeles Aqueduct Filtration Plant use ferric chloride, we initially thought of estimating tank volume using measurement tools from Google Earth. However, this proved to be quite a challenge and perhaps not the most accurate approach since the other plants may have different treatment goals.

Instead, we decided to work backwards from the annual masses of ferric chloride and emulsion polymer used specified by the Scope of Work to calculate the required tank volumes, assuming that they will be stored at the highest concentration commercially available.

## **6.5. Pump Selection**

The [Pennsylvania Department of Environmental Protection's water treatment training module](http://files.dep.state.pa.us/Water/BSDW/OperatorCertification/TrainingModules/ww07_chem_feed_wb.pdf) outlines popular chemical dosing pumps. It recommends peristaltic (tube) pumps for feed rates less than .1 gallon per hour (gph), Solenoid metering pumps for feed rates between .1 and 10 gph, and positive displacement pumps for feed rates less than 600 gallons per hour. The maximum feed rate will help decide what pump will be appropriate. 

For our design, we have selected a **positive displacement diaphragm pump for the coagulant feed** and a **solenoid metering pump for the polymer feed** based on the diffences in feed rates. Both pump types use a flexible membrane and valves to push water through the pump chamber. The membrane stretches outward to let water into the chamber and stretches inward to displace and push the water outward. While the positive displacement diaphragm pump uses a piston, plunger or diaphragm to achieve the oscillating motion. The solenoid metering pump uses an electromagnet. Shown below is a figure provided by [Diener Precision Pumps](https://www.dienerprecisionpumps.com/en/positive-displacement-pumps.html) of the pumping mechanism.

<p align="center"> 
<img src="https://www.dienerprecisionpumps.com/files/content%20bilder/info/190726/190726-diaphragm-pump.jpg">
</p>


## **6.6 Accessory & Safety Equipment** 

The Pennsylvania Department of Environmental Protection also recommends the accessory equipment for liquid feed systems. These additions will likely be inexpensive and will not contribute majorly to the plan view area of the design. For safety, we will also provide equipment to meet OSHA requirements. Note that we made several exclusions from the Pennsylvania Department of Environmental Protection's recommended list. 

Ferric chloride has a pH of about 2 and is thus very corrosive, we made sure that relevant tubing and accessories are not brass. As such, we also decided not to use a pressure relief valve as maintenance would be an issue.

We also decided not to use anti-siphon valves as we have enough redundancy and back-up storage that tanks will not be empty. Backpressure valves would not be needed based on the hydraulics of our system.

### 6.6.1. Pump Calibration Chamber
A pump calibration chamber is necessary for each pump to check the flow rate of the metering pumps used to deliver the coagulant and cationic polymer to the treatment plant. In order to maintain system accuracy, it is important to perform periodic checks of the flow rates and recalibrate the pumps as necessary. A simple calibration column can be used for this purpose, and the amount of time it tkes for the solution to fill up from the zero mark on the column to the fill mark can be divided from the volume of the column to obtain a flow rate through that pump. One thing to note though is that FeCl3 solution may clog the column over time making the calibration imperfect so the calibration columns will have to be periodically replaced or cleaned.

Shown below is a calibration column from [McMaster Carr](https://www.mcmaster.com/4446K78-4446K78) 

<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/444/4446k71-@1x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@2x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@3x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@4x_636918023428203663.png?ver=ImageNotFound">
</p>

At least one ball valve at the bottom and a return line at the top of the calibration column will also be necessary to complete the setup which will be located adjacent to the chemical storage tanks. 

### 6.6.2. Polymer Blending Unit
A polymer blending unit will be necessary to activate and mix the cationic emulsion polymer before distribution to the treatment train.

### 6.6.3. Spill Containment Zone 
Since FeCl3 is a corrosive chemical, steps will have to be taken to ensure the containment of any potential spills, according to OSHA regulations. The necessary spill containment zone must consist of grating, leak detection probes and chemical resistant coatings on the walls and floor. 

The proposed tank farm will be sunken into the ground to contain any spillage. The area will include grating as well as a drain to deal with excess chemicals. It is recommended that [chemical-resistant sorbent socks](https://www.mcmaster.com/spill-control/sorbents-for-harsh-chemicals/) be kept on hand to absorb any spills. These socks are very flexible and can cover tight corners well.

### 6.6.4. Eye Wash Stations
In the event that an operator accidentally gets a corrosive chemical like the FeCl3 coagulant in their eyes, an eye wash station should be installed at a handy location in order to prevent permanent eye damage. One such eye wash station with the necessary piping found on [McMaster Carr](https://www.mcmaster.com/eye-wash-stations/ansi-compliant-floor-mount-eye-wash-stations-with-pipe-connection/) is shown below.
<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/556/5568t13-@1x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@2x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@3x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@4x_637074372322425986.png?ver=ImageNotFoundm">
</p>

### 6.6.5. Safety Showers
Safety showers should be provided on-site to deal with larger splashes of corroseive chemicals on the operator. The one shown below combines an eyewash station and safety shower and can also be found on [McMaster Carr](https://www.mcmaster.com/5557T7-5557T7)
<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/555/55505t2c1-i01bl1571005432@1x_637065843306759702.png?ver=ImageNotFound">
</p>



# **7. Next Steps and Challenges**

1. Sketch out chemical dosing system design: 
- Select tank farm location
- Select number of tanks, pumps
- Design pipe plumbing for dosage system
2. Update the vendors table with relevant vendors. 

# **8. Design**

As the open tank farm will store ferric chloride coagulant for the rapid mix unit process as well as emulsion polymer for residuals handling, we suggest swapping the initial proposed open tank farm and truck staging areas from the layouts suggests in the Fairmont Sedimentation Plant Scope of Work document. The truck staging area may remain adjacent to the residuals handling area, while the chemicals will not have to be pumped as great of a distance to the relevant unit processes.

![plant layout](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/plant_layout.png)


![chemicals layout](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/chemicals_layout.png)

### **8.1. Maximum coagulant dosage in plant**

The first step in the design process is to determine the maximum coagulant (FeCl3) dosage expected in our system, given the design flow rate in the plant, calculated by conservation of mass: 

$$Q_{max,FeCl3} = \frac{Q_{des}C_{max,FeCl3}}{C_{stock,FeCl3}}$$
where

Q<sub>des</sub> = design flow rate of the plant = 720 cfs

C<sub>max,FeCl3</sub> = maximum coagulant dose used in the plant = 30 mg/L (assumed maximum, at least twice the average dose from Scope of Work)

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

[/Monroe/]: # (How do you know that this interpretation is correct? When I google this I get many interpretations of what a percent solution means and given the densities involved it is critical that you use the correct definition. I suggest checking with a manufacturer and include a reference to the definition of percent. This is another example of engineers using units that are unhelpful! Below you sometimes assume that the density is 1.4 g per mL and sometimes you assume the concentration is 0.4 g per mL. Those assumptions are incompatible. Figure this out!)

The necessary unit conversions have been left out of these equations as those can by easily calculated in Python using the pint library.

[Not diluting]: # (Given that the viscosity of ferric chloride is much higher than that of water, the chemicals team has decided that a dilution would be helpful for the process. Neither viscosity nor density have linear relationships with concentration; however, due to limited information about the relationship between the concentrations of ferric chloride and its density and viscosity, it is assumed that this relationship is linear. This will cause the density and viscosity of the solutions to be overestimated. We justify this assumption because both of these estimates yield a conservative design when it comes to finding the power required as well as the optimal pipe diameter. Below are the equations for the calculations for the density and viscosity of the diluted solution.)

[Not diluting]: # ($$ \rho_{FeCl_3, dil} = \rho_{FeCl_3, dil} * \frac{S_{dil}}{S_{o}} + \rho_{H_2O} * (1- \frac{S_{dil}}{S_{o}}) $$)

[Not diluting]: # ($$ \nu_{FeCl_3, dil} = \nu_{FeCl_3, dil} * \frac{S_{dil}}{S_{o}} + \nu_{H_2O} * (1- \frac{S_{dil}}{S_{o}} $$)

#### **Method 1: Scope of Work assumption**

#### **Method 2: Cottonwood Plant assumption**

Using the average dosing rate at the Cottonwood plant from [resources.md](https://github.com/MWSteams/4590FSP/blob/master/resources.md), we assumed a maximum dose tp calculate the required feed rate and 30 day storage volume required.

Since the Scope of Work assumption is more conservative, and is also more appropriate since it was determined by the Los Angeles Department of Water and Power engineers, we decided to move forward with our design using the value calculated from Method 1.

[/Monroe/]: # (Fix the incorrect assumptions for the stock concentration above.)

[/Monroe/]: # (Provide a basis for your conclusion about what type of pump to use. Note that the flow rate calculation is incorrect for multiple reasons: wrong stock concentration, wrong number of rapid mix units)

### **8.2. Storage volume of coagulant for 30 days**

[/Monroe/]: # (I believe the SOW provides an estimate of the annual use of FeCl3. I'd suggest using 1/12 of that estimate to get the storage requirement. Note that your first appoach is going around in circles because we estimated by the average coagulant dose based on the annual usage as reported in the SOW. So eliminate the first method.)

The next step is to determine the storage volume for 30 days worth of coagulant needed to run the plant. We assume that the plant is running at average conditions for this duration. The method to calculate the volume of coagulant needed over 30 days is to use the annual mass estimate of coagulant given in the scope of work, using the equation

$$V_{t,FeCl3} = \frac{M_{ann, FeCl3}}{C_{stock, FeCl3}} $$
where

M<sub>ann,FeCl3</sub> = annual estimate of mass of FeCl3 needed = 4142 tons/yr (from scope of work)

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

The SOW estimate will be used to calculate the storage since it yields the larger total feed rate required for the coagulant, ferric chloride. The 30-day storage volume required based on the SOW annual estimate is 6,100 gallons ( 230.9 kiloliter ).

[/Monroe/]: # (Include a discussion after your calculations explaining the significance of the results.)
  
To get a picture of how many tanks at the LAAFP would be required to store this volume, Google Earth was utilized to observe the existing LAAFP tank farm. Three types of tanks were identified. Below are calculations how many of each type of tank would be required to store the calculated volume. These estimates will help guide us when deciding on the number of tanks to choose.

Tank type 1 is the larger white tank on the left, which has an estimated storage capacity of 56.02 kilogallon (0.2121 megaliter). Tank type 2 is the smaller white tank on the left, which has an estimated storage capacity of 38.9 kilogallon (0.1473 megaliter).
![height of tank types 1, 2](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_1_2_height.png)

![diameters of tank types 1, 2](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_1_2_diam.png)

Tank type 3 is the large blue-green tank below, which has estimated storage capacity of 82.16 kilogallon (0.311 megaliter).
![height and diameter of tank type 3](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_3.png)



Considering that there are 2-3 tanks for each tank type in the Los Angeles Aqueduct Filtration Plant, the above estimates of having 3-6 tanks seem relatively reasonable. This suggests that we are moving in the right direction.

As mentioned previously, we would like a minimum 3 coagulant storage tanks for redundancy – one for use, one for back-up, and one for additional back-up during maintenance. However, if we keep the number of coagulant storage tanks to the minimum of 3, each tank would have to be very large. To provide more flexibility with storage tank options by having each coagulant storage tank hold a smaller volume, we have decided to move forward with 6 storage tanks for the coagulant. We also liked the number 6 as it is easily divisible since there will be up to 3 rapid mix units operating at once, and at least 2 feeds per treatment train. Additionally, as the ferric chloride coagulant is corrosive, we have decided to move forward with fiber glass tanks.

![coagulant tank](http://www.diamondfiberglass.com/wp-content/uploads/2019/06/107_0783.jpg)


### **8.3. Annual cost of coagulant**

[/Monroe/]: # (I don't have any confidence that the equation below is correct. The random ratio can't be right. And I don't know where you got the cost or what the units on the cost are. Is this cost per lb of FeCl3 or is it cost per lb of solution?)

[Research](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144) into vendors showed that for bulk orders, the cost of 40% FeCl3 solution is around $0.2243/lbs. This value can be used to calculate the annual cost of coagulant using the formula

$$Annual\: cost\: of\:FeCl3 = (unit\: cost\:of\:FeCl3)*V_{30,FeCl3}* \rho_{FeCl3}$$

where 

V<sub>30,FeCl3</sub> = Volume needed to store 30 days worth of 40% FeCl3 solution (calculated above)

$\rho$<sub>FeCl3</sub> = density of 40% FeCl3 solution = 1.356 g/mL ( [BrennTag Connect](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144))

[/Monroe/]: # (Here you give a difference density of the stock solution. Getting these numbers right is a first step in your design. Make sure to fix this asap.)

[/Monroe/]: # (I believe the SOW include a cost for the coagulant. Does your calculation agree?)

The answer calculated above is roughly 3 times the amount estimated in the scope of work. Usually, budget predictions are conservative which makes the ratio result surprising. Since we do not have a better idea of where the SOW price estimate comes from, we will leave our calculations as is. It is possible that LA engineers have a better unit pricing than the price posted online. 

### **8.4. Storage Volume & Max. Feed Rate of Emulsion Polymer**

Calculating the storage volume and maximum feed rate of the emulsion polymer is done similarly to the calculations used for ferric chloride. 

The maximum flow rate will be calculated using conservaion of mass. Knowing the concentration of the stock polymer solution, the concentration desired in the treatment train, and the maximum flow of the plant will be needed to calculate the maximum flow rate for the polymer stock solution. The equation utilized is provided below:  

$$Q_{max,polymer} = \frac{Q_{des}C_{max,polymer}}{C_{stock,polymer}}$$

The volume needed to store polymer solution for 30 days is provided below. It is based on the definitions of volume, solution concentration, and mass. 

$$V_{month,polymer} = \frac{M_{month, polymer}}{C_{stock, polymer}} $$

The Tramfloc 300-399 Series Cationic Emulsion Polymer was selected for this design as a representative polymer. The polymer's safety data sheet may be found [here](http://tramfloc.com/material-safety-data-sheets/).




The storage volume and maximum feed rate of the emulsion polymer may be done either by using the annual polymer mass estimate from the SOW or the average dose of polymer used at the Cottonwood Plant.

#### **Method 1: Scope of Work assumption**

The Scope of Work assumption was what was used to calculate the volume of coagulant needed for 30 days of storage. 

#### **Method 2: Cottonwood Plant assumption**

Using the average dosing rate at the Cottonwood plant from [resources.md](https://github.com/MWSteams/4590FSP/blob/master/resources.md), we assumed a maximum dose tp calculate the required feed rate and 30 day storage volume required.

The two methods used to calculate emulsion polymer storage volume requirements are drastically different. The Scope of Work assumption gave a 30-day storage volume estimate of 12.35 kiloliters, while the Cottonwood assumption gave an estimate of 7986 kiloliters. The results from the Scope of Work method appear to be in a reasonable magnitude whereas the results from the Cottonwood Plant method appears to be an extremely high and unreasonable value. One of the reasons this could be the case is because there is no residuals handling treatment train at Cottonwood, so the polymer they use would be for a different purpose.

Since the Scope of Work was likely already a conservative estimate used by the Los Angeles Department of Water and Power to calculate annual costs, and 12.35 kiloliters is a reasonable value, we will be using this value as our 30-day storage volume for the emulsion polymer.

### **8.5. Pipe Design**

[Chemicals]: # (TBC: Need to connect this to the pump selection process as well as incorporate this into the pipe diameter selection from AC Python Package)

[Monroe]: # (Okay, you missed this last time. If you have questions about how to do these calculations, bring me in to a meeting!)

[Monroe]: # (Design for the maximum chemical flow rate. It doesn't make sense to design a chemical feed system that fails when the flow rate is greater than the average flow. Don't forget to include the number of chemical dosing locations.)

The design below calculates the optimal nominal diameter for a schedule 40 pipe. First, the function ac.diam_pipe was used to find the optimal inner diameter. This is then used to find the corresponding schedule 40 inner diameter and nominal diameter. The inputs the function considered where: the pipe flow, the acceptable head loss, the pipe length, the coagulant viscosity, the roughness of the pipe, and the minor losses. 

The design will include 2 feed lines per rapid mix unit for redundancy purposes. It is also assumed that 1 m of head loss is acceptable to start. 

### **8.6 Pump Design**

Given that there are 4 rapid mix units and 1 residuals unit in this plant, and that for redudancy we would like 2 feed systems into each unit, we will have a total of 8 pump for the coagulant feed system and 2 pumps for the polymer feed system. 

Pump selection will be made based on the maximum flow rates required per feed train. This is 484.7 gal/hour (509.7 mL/s) for the coagulant and and 4.466 gal/hour (4.696 mL/s) for the emulsion polymer.

The threshold for a solenoid pump is 10 gph while the the threshold for a positive displacement diaphragm pump is 600 gph. 

### **8.7 Calibration Chamber**

We will need two types of pump calibration chamnbers. The first type is for the coagulant feed, the second type is for the polymer feed. The calibration chambers will be sized such that it takes at least 30 seconds for a reading at the maximum chemical flow rate.

$$t_{fill} = 30 s$$

The minimum volume of the calibration chamber for the coagulant feed pumps is determined by the following:

$$V_{min,FeCl3} = {Q_{max,FeCl3} * t_{fill}} $$

Similarly, to calculate the minimum volume of the calibration chamber for the polymer feed:

$$V_{min,polymer} = {Q_{max,polymer} * t_{fill}} $$

This method results in a minimum calibration chamber volume of 15,290 mL for the coagulant feed and 140.9 mL for the polymer feed. The chosen vendor is sufficient based on these requirements. We have selected the models with 20,000 mL and 250 mL volume capacity.




# **Vendors and contact information file**

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



## **Appendix**

In [41]:
!pip install aguaclara
!pip install molmass

In [42]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-05-18 21:00:17--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3064 (3.0K) [text/plain]
Saving to: ‘Fairmont.py.6’

Fairmont.py.6       100%[===================>]   2.99K  --.-KB/s    in 0s      

2020-05-18 21:00:17 (44.3 MB/s) - ‘Fairmont.py.6’ saved [3064/3064]



In [43]:
# List of vendors compiled by all teams in CEE 4590
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design
5,Pump calibration columnn,McMaster Carr,NaN,NaN,https://www.mcmaster.com/4446K78-4446K78,https://www.mcmaster.com/4446K78-4446K78,5/15/2020,Ananya Gangadhar,"20,000 mL for FeCl3 and 250 mL for polymer"
6,Eye wash stations,McMaster Carr,NaN,NaN,https://www.mcmaster.com/eye-wash-stations/ans...,https://www.mcmaster.com/5568T13-5568T13,5/15/2020,Ananya Gangadhar,NaN
7,...EDIT sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
8,..EDIT plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
9,rapid mixer (mech),Dynamix,still,pending,https://www.dynamixinc.com,https://www.dynamixinc.com/industries/waste-wa...,4/16/2020,Lynn Li,NaN


In [44]:
# Relevant parameters related to ferric chloride stock solution: 
stock_strength_fecl3 = 0.4 # Active strength of ferric chloride stock solution
stock_density_fecl3 = 1.356*u.g/(u.mL) # for 40% m/v FeCl3, Brenntech Connect 
stock_c_fecl3 = 400 * u.g/u.L # conc of coagulant stock in tank
stock_mu_fecl3 = 0.12 * u.N * u.s / u.m**2 # for 40% m/v FeCl3, Brenntech Connect
stock_nu_fecl3 = stock_mu_fecl3/stock_density_fecl3
temp = 20 *u.degC

# Relevant parameters related to diluted ferric chloride feed solution:
feed_strength_fecl3 = stock_strength_fecl3 # change if it will be diluted
feed_density_fecl3 = stock_density_fecl3*feed_strength_fecl3/stock_strength_fecl3 + ac.density_water(temp)* (1 - feed_strength_fecl3/stock_strength_fecl3)
feed_nu_fecl3 = stock_nu_fecl3*feed_strength_fecl3/stock_strength_fecl3 + ac.viscosity_kinematic_water(temp)* (1 - feed_strength_fecl3/stock_strength_fecl3)

# Coagulant stock flow (max, total) - SOW
stock_q_fecl3 = (fmt.q_max * fmt.Fe_c_max/stock_c_fecl3).to(u.mL/u.s)
print('The total maximum ferric chloride dosing flow rate is',stock_q_fecl3.to(u.gal/u.hour), "(",stock_q_fecl3.to(u.mL/u.s),")")

# Coagulant stock flow (max, total) - Hering & Kneebone 2002 (p.169)
fecl3_dose_avg_CTP = 5.7 * u.mg/u.L 
fecl3_dose_max_CTP = fecl3_dose_avg_CTP * 2 # Assume that maximum dose is double
fecl3_mass_delivery_rate_CTP = (fmt.q_max * fecl3_dose_max_CTP)/stock_strength_fecl3
fecl3_vol_delivery_rate_CTP = fecl3_mass_delivery_rate_CTP/stock_density_fecl3
print("The total maximum feed rate of ferric chloride solution is", fecl3_vol_delivery_rate_CTP.to(u.gal/u.hour), "(",fecl3_vol_delivery_rate_CTP.to(u.mL/u.s),")")

# Coagulant feed rate (max, per train) 
feeds_per_train = 2 #two pipes for redundancy
num_feed_pipes = feeds_per_train*fmt.n_floc_mech #total # coagulant feed pipes
feed_q_tot_fecl3 = stock_q_tot_fecl3* stock_strength_fecl3/feed_strength_fecl3 #total feed Q of coagulant in plant
train_q_water = fmt.q_max/(fmt.n_floc_mech-1) #Q of water in each treatment train
print("The maximum flow rate for the raw water per train is", train_q_water.to(u.gal/u.hour), "(",train_q_water.to(u.mL/u.s),")")
train_q_fecl3 = feed_q_tot_fecl3 / (fmt.n_rm_mech-1) 
print("The maximum flow rate for the coagulant feed per train is", train_q_fecl3.to(u.gal/u.hour), "(",train_q_fecl3.to(u.mL/u.s),")")
t_fill = 30 * u.second
cal_chamber_fecl3 = train_q_fecl3 * t_fill
print("The minimum volume of the pump calibration chamber for coagulant feed is", cal_chamber_fecl3.to(u.mL))

# Coagulant volume for 30 days - SOW 
ann_mass_fecl3 = 4142 * u.tons #provided by SOW
month_mass_fecl3 = ann_mass_fecl3 *u.month/u.year 
month_vol_fecl3 = month_mass_fecl3 /stock_density_fecl3
print("The volume of FeCl3 needed for 30 days based on the SOW annual estimate is", month_vol_fecl3.to(u.kgal), "(",month_vol_fecl3.to(u.kL),")")

# Coagulant annual cost
SOW_ann_cost_fecl3 = 2100000 *u.USD #from SOW estimate
unit_cost_fecl3 = 0.2243 * u.USD/u.lbs # from Brenntag vendor
ann_cost_fecl3 = unit_cost_fecl3*ann_mass_fecl3
cost_ratio = ann_cost_fecl3/SOW_ann_cost_fecl3
print('The annual cost of FeCl3 from the selected vendor is', ann_cost_fecl3.to(u.USD))
print("The cost ratio based on the vendor is", cost_ratio.to(u.dimensionless).magnitude, "the cost based on the scope of work")

# Relevant paramater of polymer stock solution:
density_polymer = 1.01*u.g/(u.mL) # Tramfloc 300-399 Series Cationic Emulsion Polymer
polymer_strength = 0.35 # Active strength of polymer solution
polymer_stock = 5 *u.g/u.L 
mu_polymer = 1 * u.N * u.s / u.m**2 
nu_polymer = mu_polymer/density_polymer

# Polymer volume for 30 days & feed rate
ann_mass_polymer = 165 * u.tons # provided by SOW
month_mass_polymer = ann_mass_polymer *u.month/u.year
month_vol_polymer = month_mass_polymer/density_polymer
q_polymer = month_vol_polymer / u.month
print("The volume needed to store polymer for 30 days is", month_vol_polymer.to(u.kgal), "(",month_vol_polymer.to(u.kL),")" )
print("The maximum flow rate of the polymer solution is", q_polymer.to(u.gal/u.hour), "(", q_polymer.to(u.L/u.sec),")" )
cal_chamber_polymer = q_polymer * t_fill
print("The minimum volume of the pump calibration chamber for polymer feed is", cal_chamber_polymer.to(u.mL))

# function to find optimal sch40 ND:
def ND_sch40(Q, head_l, pipe_l, nu, e, minor_loss): 
  diam_pipe = ac.diam_pipe(Q, head_l, pipe_l, nu, e, minor_loss)
  av_pipe_array = ac.ID_sch40(ac.ND_all_available()) #returns all ID for sch40
  ID_sch40 = ac.ceil_nearest(diam_pipe, av_pipe_array)
  index = np.where(ID_sch40 == av_pipe_array) #index of ND and ID
  ND_array = ac.ND_all_available()
  ND_sch40 = ND_array[index][0]
  return ND_sch40

def Hl_actual(Q, head_l, pipe_l, nu, e, minor_loss): #if you're willing to have this amount of hl, design is okay.
  diam_pipe = ac.diam_pipe(Q, head_l, pipe_l, nu, e, minor_loss)
  av_pipe_array = ac.ID_sch40(ac.ND_all_available()) #returns all ID for sch40
  ID_sch40 = ac.ceil_nearest(diam_pipe, av_pipe_array)
  hl_actual = ac.headloss_pipe(Q, ID_sch40, pipe_l, nu, e, minor_loss)
  return hl_actual

# Relevant pipe parameters:
e = 0.0015 *u.mm #Frank Fluid Mechanics pg 371, assumed plastic PVC pipe
k_elbow = 0.3 #Frank Fluid Mechanics pg 391, assumed 90 deg. regular flanged 4 in. nominal diameter

# Coagulant Feed Pipe: optimal sch40 ND
feed_pipe_l = 1000 *u.feet 
feed_pipe_elbow = 4
feed_pipe_hl = 10 *u.m #1 m headloss to start
feed_ND_sch40 = ND_sch40(train_q_fecl3, feed_pipe_hl, feed_pipe_l, feed_nu_fecl3, e, feed_pipe_elbow*k_elbow)
feed_hl_actual = Hl_actual(train_q_fecl3, feed_pipe_hl, feed_pipe_l, feed_nu_fecl3, e, feed_pipe_elbow*k_elbow)
print("The nominal diameter of the coagulant feed pipe is",feed_ND_sch40)
print("The actual headloss of the coagulant feed pipe design is", feed_hl_actual) 

# Coagulant Stock pipe: optimal sch40 ND
stock_pipe_l = 200 *u.feet 
stock_pipe_elbow = 4
stock_pipe_hl = 10 *u.m #1 m headloss to start
stock_ND_sch40 = ND_sch40(stock_q_fecl3, stock_pipe_hl, stock_pipe_l, stock_nu_fecl3, e, stock_pipe_elbow*k_elbow)
stock_hl_actual = Hl_actual(stock_q_fecl3, stock_pipe_hl, stock_pipe_l, stock_nu_fecl3, e, stock_pipe_elbow*k_elbow)
print("The nominal diameter of the coagulant stock pipe is",stock_ND_sch40)
print("The actual headloss of the coagulant stock pipe design is", stock_hl_actual)

# Polymer Stock pipe: optimal sch40 ND
polymer_pipe_l = 500 *u.feet 
polymer_pipe_elbow = 4
polymer_pipe_hl = 10 *u.m #1 m headloss to start
polymer_ND_sch40 = ND_sch40(q_polymer, polymer_pipe_hl, polymer_pipe_l, nu_polymer, e, polymer_pipe_elbow*k_elbow)
polymer_hl_actual = Hl_actual(q_polymer, polymer_pipe_hl, polymer_pipe_l, nu_polymer, e, polymer_pipe_elbow*k_elbow)
print("The nominal diameter of the polymer stock pipe is",polymer_ND_sch40)
print("The actual headloss of the polymer stock pipe design is", polymer_hl_actual)

# Coagulant Water Dilution pipe: optimal sch40 ND
# fw_pipe_l = 20 *u.feet 
# fw_pipe_elbow = 4
# fw_pipe_hl = 1 *u.m #1 m headloss to start
# fw_ND_sch40 = ND_sch40(train_q_fecl3, fw_pipe_hl, fw_pipe_l, ac.viscosity_kinematic_water(temp), e, fw_pipe_elbow*k_elbow)
# fw_hl_actual = Hl_actual(train_q_fecl3, fw_pipe_hl, fw_pipe_l, ac.viscosity_kinematic_water(temp), e, fw_pipe_elbow*k_elbow)
# print("The nominal diameter of the water pipe for polymer dilution is",polymer_ND_sch40)

# Polymer Water Dilution pipe: optimal sch40 ND
pw_pipe_l = 100 *u.feet 
pw_pipe_elbow = 4
pw_pipe_hl = 10 *u.m #1 m headloss to start
q_pw = 600 *u.gal/u.hour # max water flow rate for PolyBlend PB 600 model
pw_ND_sch40 = ND_sch40(q_pw, pw_pipe_hl, pw_pipe_l, ac.viscosity_kinematic_water(temp), e, pw_pipe_elbow*k_elbow)
pw_hl_actual = Hl_actual(q_pw, pw_pipe_hl, pw_pipe_l, ac.viscosity_kinematic_water(temp), e, pw_pipe_elbow*k_elbow)
print("The nominal diameter of the water pipe for polymer dilution is",polymer_ND_sch40)

# Pump power requirements and cost calculations

def power_pump(density, Q, hl, elevation_i, elevation_f): 
  head_pump = hl + elevation_f - elevation_i
  power_pump = head_pump * ac.GRAVITY * Q * density
  return power_pump

coagulant_elevation_tank = 2 *u.feet #reference elevation
polymer_elevation_tank = 2 *u.feet #reference elevation
water_elevation = -2 * u.feet
residual_elevation = 0 *u.feet #lower elevation, 0 is conservative design
rapid_mix_elevation = 10 *u.feet #max. elevation change based on SOW elevation map 
electricity_cost = .12*u.USD/(u.kW*u.hour)

feed_pump_power = power_pump(feed_density_fecl3, train_q_fecl3, feed_hl_actual, coagulant_elevation_tank, rapid_mix_elevation)
polymer_pump_power = power_pump(density_polymer, q_polymer, polymer_hl_actual, polymer_elevation_tank, residual_elevation)
fw_pump_power = power_pump(ac.density_water(temp), train_q_fecl3, fw_hl_actual, water_elevation, coagulant_elevation_tank)
pw_pump_power = power_pump (ac.density_water(temp), q_pw, pw_hl_actual, water_elevation, polymer_elevation_tank)

pump_power_total = polymer_pump_power + pw_pump_power + (feed_pump_power + fw_pump_power)* (fmt.n_floc_mech - 1) 
ann_cost_pump_power = pump_power_total * electricity_cost *u.year
print("The total power required to deliver the polymer and coagulant to their respective treatment trains is", pump_power_total.to(u.watt))
print("The annual cost of power for the coagulant and polymer pumps is",ann_cost_pump_power.to(u.USD))

The total maximum ferric chloride dosing flow rate is 1454 gallon / hour ( 1529 milliliter / second )
The total maximum feed rate of ferric chloride solution is 407.5 gallon / hour ( 428.5 milliliter / second )
The maximum flow rate for the raw water per train is 2.77e+06 gallon / hour ( 2.913e+06 milliliter / second )
The maximum flow rate for the coagulant feed per train is 484.7 gallon / hour ( 509.7 milliliter / second )
The minimum volume of the pump calibration chamber for coagulant feed is 1.529e+04 milliliter
The volume of FeCl3 needed for 30 days based on the SOW annual estimate is 61 kilogallon ( 230.9 kiloliter )
The annual cost of FeCl3 from the selected vendor is 1.858e+06 dollar
The cost ratio based on the vendor is 0.8848100952380953 the cost based on the scope of work
The volume needed to store polymer for 30 days is 3.263 kilogallon ( 12.35 kiloliter )
The maximum flow rate of the polymer solution is 4.466 gallon / hour ( 0.004696 liter / second )
The minimum volume of

In [45]:
# Utilizing go-by's to get compare volume needed and typical tank sizes

#LAAFP tank volume for type 1 (larger white tanks on the left)
LAAFP_tank_1_height = 7.5 * u.m
LAAFP_tank_1_diam = 6 * u.m
LAAFP_tank_1_volume = np.pi * (LAAFP_tank_1_diam/2)**2 * LAAFP_tank_1_height
LAAFP_tank_1_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_1_volume.to(u.L))
print("The volume estimate for LAAFP tank type 1 is",LAAFP_tank_1_volume.to(u.kilogallon), "(",LAAFP_tank_1_volume.to(u.ML),")")
print((LAAFP_tank_1_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 1 to store", month_vol_fecl3.to(u.kL), "(",month_vol_fecl3.to(u.ML),")")

#LAAFP tank volume for type 2 (smaller white tanks on the right)
LAAFP_tank_2_height = 7.5 * u.m
LAAFP_tank_2_diam = 5 * u.m
LAAFP_tank_2_volume = np.pi * (LAAFP_tank_2_diam/2)**2 * LAAFP_tank_2_height
LAAFP_tank_2_total = LAAFP_tank_2_volume * 3
LAAFP_tank_2_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_2_volume.to(u.L))
print("The volume estimate for LAAFP tank type 2 is", LAAFP_tank_2_volume.to(u.kilogallon),"(",LAAFP_tank_2_volume.to(u.ML),")")
print((LAAFP_tank_2_number.to(u.dimensionless)).magnitude,"tanks are required for tank type 2 to store", month_vol_fecl3.to(u.kL),  "(",month_vol_fecl3.to(u.ML),")")

#LAAFP tank volume for type 3 (blue-green tanks)
LAAFP_tank_3_height = 11 * u.m
LAAFP_tank_3_diam = 6 * u.m
LAAFP_tank_3_volume = np.pi * (LAAFP_tank_3_diam/2)**2 * LAAFP_tank_3_height
LAAFP_tank_3_total = LAAFP_tank_3_volume * 2
LAAFP_tank_3_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_3_volume.to(u.L))
print("The volume estimate for LAAFP tank type 3 is", LAAFP_tank_3_volume.to(u.kilogallon),"(",LAAFP_tank_3_volume.to(u.ML),")")
print((LAAFP_tank_3_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 3 to store", month_vol_fecl3.to(u.kL),  "(",month_vol_fecl3.to(u.ML),")")

The volume estimate for LAAFP tank type 1 is 56.02 kilogallon ( 0.2121 megaliter )
2.0 tanks are required for tank type 1 to store 230.9 kiloliter ( 0.2309 megaliter )
The volume estimate for LAAFP tank type 2 is 38.9 kilogallon ( 0.1473 megaliter )
2.0 tanks are required for tank type 2 to store 230.9 kiloliter ( 0.2309 megaliter )
The volume estimate for LAAFP tank type 3 is 82.16 kilogallon ( 0.311 megaliter )
1.0 tanks are required for tank type 3 to store 230.9 kiloliter ( 0.2309 megaliter )
